資料預處理

In [1]:
# Pre-process
train_data = pd.read_csv('./adult/adult.data', header= None, names=['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'class'])
test_data = pd.read_csv('./adult/adult.test', header= None, names=['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'class'])
originTest = test_data

NameError: name 'pd' is not defined

In [ ]:
# education and education-num have same meaning
train_data.drop(['education'], axis = 1, inplace = True)
test_data.drop(['education'], axis = 1, inplace = True)

# fnlwgt is not important feature
train_data.drop(['fnlwgt'], axis = 1, inplace = True)
test_data.drop(['fnlwgt'], axis = 1, inplace = True)

# remove the space
train_data = train_data.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
test_data = test_data.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [ ]:
train_data.apply(lambda x: np.sum(x == "?"))

age                  0
workclass         1836
education-num        0
marital-status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     583
class                0
dtype: int64

In [ ]:
# replace the "?" into NAN
train_data.replace("?", pd.NaT, inplace = True)
test_data.replace("?", pd.NaT, inplace = True)

# replace the NAN into mode value
train_data['workclass'] = train_data['workclass'].replace(float('nan'), train_data['workclass'].mode()[0])
train_data['occupation'] = train_data['occupation'].replace(float('nan'),train_data['occupation'].mode()[0])
train_data['native-country'] = train_data['native-country'].replace(float('nan'),train_data['native-country'].mode()[0])

# tranfer the value of class(income) into int(1 or 0)
# >50K is 1, <=50K is 0
train_data['class'] = train_data['class'].apply(lambda x: 1 if x == ">50K" else 0)
test_data['class'] = train_data['class'].apply(lambda x: 1 if x == ">50K" else 0)

In [ ]:
train_data.head()

,age,workclass,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


In [ ]:
#One Hot Encoding (Dummies)

trainData_dum = pd.get_dummies(train_data, columns=['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'], dtype=int)
testData_dum = pd.get_dummies(test_data, columns=['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'], dtype=int)

#pd.DataFrame(trainData_dum)

In [ ]:
# from openpyxl import Workbook

# trainData_dum.to_excel("TraindataDum.xlsx", sheet_name="TraindataDum", index=False)
# testData_dum.to_excel("TestdataDum.xlsx", sheet_name="TestdataDum", index=False)

In [ ]:
trainData_dum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 89 columns):
 #   Column                                     Non-Null Count  Dtype
---  ------                                     --------------  -----
 0   age                                        32561 non-null  int64
 1   education-num                              32561 non-null  int64
 2   capital-gain                               32561 non-null  int64
 3   capital-loss                               32561 non-null  int64
 4   hours-per-week                             32561 non-null  int64
 5   class                                      32561 non-null  int64
 6   workclass_Federal-gov                      32561 non-null  int32
 7   workclass_Local-gov                        32561 non-null  int32
 8   workclass_Never-worked                     32561 non-null  int32
 9   workclass_Private                          32561 non-null  int32
 10  workclass_Self-emp-inc                     325

In [ ]:
from sklearn.preprocessing import MinMaxScaler

#在讀熱編碼後會依照有名目之欄位產生資料，train_data比test_data多出了該欄位，故將test_data新增該欄位，讓兩個資料集欄位相同。
testData_dum['native-country_Holand-Netherlands'] = 0

#刪除重複列
trainData_dum.drop_duplicates(inplace=True)
testData_dum.drop_duplicates(inplace=True)

trainData_dum.describe()

,age,education-num,capital-gain,capital-loss,hours-per-week,class,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
count,28970.000000,28970.000000,28970.00000,28970.000000,28970.000000,28970.000000,28970.000000,28970.000000,28970.000000,28970.000000,...,28970.000000,28970.000000,28970.000000,28970.000000,28970.000000,28970.000000,28970.000000,28970.000000,28970.000000,28970.000000
mean,39.280428,10.100759,1203.01184,97.597825,40.652054,0.247843,0.032654,0.070349,0.000242,0.729375,...,0.001277,0.003935,0.000414,0.002761,0.001760,0.000621,0.000656,0.903521,0.002313,0.000552
std,13.682949,2.647603,7794.72045,424.880797,12.748275,0.431767,0.177734,0.255738,0.015543,0.444290,...,0.035715,0.062608,0.020349,0.052478,0.041921,0.024919,0.025602,0.295252,0.048036,0.023495
min,17.000000,1.000000,0.00000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,28.000000,9.000000,0.00000,0.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,38.000000,10.000000,0.00000,0.000000,40.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,48.000000,13.000000,0.00000,0.000000,45.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,90.000000,16.000000,99999.00000,4356.000000,99.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


CART

In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier

from operator import itemgetter
# import copy
# import random
# import statistics

In [ ]:
def get_thresholds(index,dataset): #dataset sent in here should be already sorted 
    thresholds = []
    thresholds.append(dataset[0][index]-1) #first threshold
    for i in range(len(dataset)):
        if(i!=0): #if not the first row
            prev = dataset[i-1][index]
            cur = dataset[i][index]
            if(prev!=cur):
                thresholds.append((prev+cur)/2) #add the mean as one threshold
    thresholds.append(dataset[-1][index]+1) #last threshold
    return thresholds

#把資料分成左子樹，右子樹
def data_split(index, value, dataset): 
	left,right = list(),list()
	for row in dataset:
		if(row[index] < value):
			left.append(row)
		else:
			right.append(row)
	return left, right

In [ ]:
def gini_index(groups): #return the best threshold for that index
    gini = 0.0 #of the threshold
    for group in groups:
        if(len(group)==0): #avoid divided by zero
            continue
        pos_y = 0 #how many positive labels in the group
        for row in group:
            if(row[-1]==1):
                pos_y += 1
        neg_y = len(group) - pos_y
        gini_index = (1 - (pos_y/len(group))**2 - (neg_y/len(group))**2) 
        gini += len(group) * gini_index #to weight different lengths differently
    return gini

def data_same(original_dataset):
    dataset = copy.deepcopy(original_dataset)
    for row in dataset: 
        row.pop() #把label拿掉
    allsame = True
    for i in range(len(dataset)):
        if(dataset[0]!=dataset[i]):
            allsame = False
    return allsame

In [ ]:
class Node:
    def __init__(self,index,threshold):
        self.left = None 
        self.right = None
        self.index = index
        self.label_l = 0
        self.label_r = 0
        self.threshold = threshold
    def set_left(self,aNode):
        self.left = aNode
    def set_right(self,aNode):
        self.right = aNode
    def set_label(self,best_groups): 
        if(self.left == None and self.right == None):#沒有子節點的代表是最下方的節點，標示出他們的label
            l_sum,r_sum = 0,0
            
            for row in best_groups[0]: #左節點
                l_sum += row[-1]
            if(l_sum>0): #如果+1比-1還要多的話，標示為+1(投票機制)
                self.label_l = 1
            else:
                self.label_l = -1
                
            for row in best_groups[1]: #右節點
                r_sum += row[-1]
            if(r_sum>0): #如果+1比-1還要多的話，標示為+1(投票機制)
                self.label_r = 1
            else:
                self.label_r = -1

In [ ]:
def get_split(dataset):
    best_index, best_threshold, best_gini, best_groups = 9999, 9999, 9999, None
    terminal = False #判斷是否終止
    for index in range(10):
        sorted_data = sorted(dataset, key=itemgetter(index))
        thresholds = get_thresholds(index,sorted_data)
        for threshold in thresholds:
            groups = data_split(index,threshold,sorted_data)
            gini = gini_index(groups)
            if(gini<best_gini):
                best_gini = gini
                best_index,best_threshold,best_gini,best_groups = index,threshold,gini,groups
    aNode = Node(best_index,best_threshold)

    if(best_gini==0 or data_same(dataset)): #終止條件判斷
        terminal = True
    
    if(terminal):
        aNode.set_label(best_groups)
        return aNode
    else:
        aNode.set_left(get_split(best_groups[0]))
        aNode.set_right(get_split(best_groups[1]))
        return aNode

In [ ]:
def prediction(dataset,aNode):
    index, threshold = aNode.index, aNode.threshold
    #先依index排序，再split成兩組
    sorted_data = sorted(dataset, key=itemgetter(index))
    groups = data_split(index,threshold,dataset)
    err = 0
    results = []
    if(aNode.left == None): # it doesn't have any child node, we should do the classification
        #左節點
        for row in groups[0]:
            results.append((row,aNode.label_l))
            if(row[-1]!=aNode.label_l):
                err += 1
        #右節點
        for row in groups[1]:
            results.append((row,aNode.label_r))
            if(row[-1]!=aNode.label_r):
                err += 1
    
        return err,results

    else: # the node has child nodes, keep traversing
        err_l,results_l = prediction(groups[0],aNode.left)
        err_r,results_r = prediction(groups[1],aNode.right)
        err += err_l+err_r #add the classification error of two child nodes to err

        for element in results_l:
            results.append(element)
        for element in results_r:
            results.append(element)
        return err,results

In [ ]:
# put "class" to the last column
td = trainData_dum
column = td.pop("class")
td.insert(td.shape[1], "class", column)

columns = td.columns.tolist()

X_train = td.drop('class', axis=1)
y_train = td['class']

ted = testData_dum
t_column = ted.pop("class")
ted.insert(ted.shape[1], "class", t_column)

X_test = ted.drop('class', axis=1)
y_test = ted['class']


#CART
cart1 = get_split(td)
true_pred, preds = prediction(td, cart1)

print('Full set test accuracy:', true_pred / len(X_test))

# print(preds)

TypeError: unsupported operand type(s) for -: 'str' and 'int'

In [ ]:
#將 R 的預測結果轉換為 Python 陣列
#rpy2
from openpyxl import Workbook
#產出Excel(Test data)
wb = Workbook()
ws = wb.active
ws.append(['age', 'workclass', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country','income','Predict result'])

for i in range(len(preds)):
    if preds[i] == 0:
        result = '<=50K.'
    else:
        result = '>50K.'
    #將現在loop到原始資料的列轉為list
    li = originTest.iloc[i,:].tolist()
    #
    li.append(result)
    ws.append(li)
wb.save('C45.xlsx')
# predictions = np.array(predictions)

In [ ]:
print("Total Node：", len(system_test.tree_nodes))
leaf_count = 0
for n in system_test.tree_nodes:
    # print(n.print_node())
    if n.node_type == 'leaf':
        leaf_count += 1

print("Leaf Node：", leaf_count)

Total Node： 136
Leaf Node： 65


In [ ]:
# linear regression feature importance
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot

In [ ]:
# get importance
importance = system_test.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()

In [ ]:
import matplotlib.pyplot as plt

#以圖表事每個特徵變數的重要程度(0最小，1最大)
def plot_feature_importances_cancer(model):
    n_features = X_train.shape[1]
    plt.figure(figsize=(15,30))
    plt.barh(np.arange(n_features), model.feature_importances_, align='center')
    plt.yticks(np.arange(n_features), train_x.columns)
    plt.xlabel("Feature importance")
    plt.ylabel("Feature")
    plt.ylim(-1, n_features)

plot_feature_importances_cancer(system_test)